## Objective:


*   Clean facility name (remove special characters, capitalize letters, convert roman numerals ...)
*   Correct misspelling with help of spelling dictionary
*   Separate facility name and facility type into separate columns with help of type dictionary
*   Create facility sub-types



#Process


### Step 1: 
Make a copy of this script and add the name of your country to the file name, e.g. preprocessing_MOZ. Then, run this notebook on your health facility datasets. Doing so will add the following columns to your health facility datasets:
* **facility_name_short**: Cleaned facility names without a type. An example entry is 'Columbia,' which has been converted from 'columbia hospital.'
* **extract_type** :These are raw facility types extracted from **facility_name** column. 
* **sub_type** : Formatted and aggregated facility type from **extract_type** column. The type dictionary is going to be used as reference facility types. In some countries, some facilities are written as District Hospital, and some of them as Hospital District. The **extract_type** column is going to hold both types, but in **sub_type** we will just keep District Hospital (based on type dictionary) as it is but reformat Hospital District as District Hospital.
* **score** : We are using a fuzzy match method to aggregate and format facility types from the **extract_type** column. This column shows the match score, which is a number 100-0. 100 indicates a perfect match, and 0 indicates that the values are totally different. 
* **hf_name_length** : Length of facility name. 
* **hf_type_length** :  Length of facility type. 
* **special_chrs** : List of special characters that a facility name includes such as !?\/`"&* 
* **only_numerical** : Facility names that consist of only numerical values



### Step 2:

* **Go through each facility type in the sub_type column that has a score lower than 75**. We want to make sure we  aggregate facility types in the right way. A score lower than 75 can be misclassified. Correct the cases that you find missclassified in the **sub_type** column.
* **Check facilities without a type**. Some of the facilites do not have a type. For those facilities, the **sub_type** column should be empty. For these cases, check the **facility_name_short** column and confirm facility name does not include a type. If you find that facility name includes its type, manually exclude facility type from facility name in **facility_name_short** column and move facility type to **sub_type** column. 
* **Check facilities without a name**. Some of the facilities do not have a name. For those cases **facility_name_short** should be empty. If it's empty, check **facility_name** column and confirm that the facility name does not include a name (just type). If you find that, in the **facility_name** column, a facility name exists, manually enter facility name into **facility_name_short** column.
* **Check very long or short facility name**. Some facility names are very long or short. Sometimes that's an indication that the name is wrong, so we need to manually check. Please go over facility names in *facility_name_short* column  that have over 30 characters and less than 3 characters. You can sort text by length in Excel. Correct facility name in **facility_name_short** column if you think they are misspelled. 
* **Check very long or short facility types**: Some of the facility types are very long or short.  Please go over facility types in **sub_type** that have over 20 characters and fewer than 3 characters. Correct facility type at **sub_type**  if you think they are misspelled. 
* **Check facility names that have a special characters**.Facility names that have special characters have been palced in a  **special_chrs** column.Some of these characters are not right.Remove the characters  at **facility_name_short** column  that are listed **special_chrs** column  that you think wrong.
* **Check facility names that consist of only numerical characters**. These facilities are flagged in an **only_numerical** column. For these cases, check **facility_name**  column name, and confirm that facility name consists of only numerical values. If not, correct the facility name in the **facility_name_short** column.



### Step 3:

* Repeat steps 1 and 2 for each health facility dataset you have explored in previous landscaping exercises. 







# Part 1 : Import Modules


The cell below imports some modules that are not existent in python's default enviroment. You need to run the cell below to import external modules into the notebook environment. Just run the cell without modifying it. Run the cell by hovering over the cell and clicking the icon in the top left. When it is finished, a green checkmark will appear to the left of the cell.

In [ ]:
!pip install Ordered_set
!pip install thefuzz

The cell below imports required modules. Just run the cell without modifying it. It does not create any output. It runs very fast. 


In [ ]:
import pandas as pd
import re
import numpy as np
from google.colab import files
import io
from ordered_set import OrderedSet
from thefuzz import process, fuzz



# Part 2: Provide required inputs

The cell below requires some inputs from you. Please provide required inputs as explained below and then click the run icon on the top left of the cell:


*   **country name** : Name of country (either iso code or full name)
*   **health_facility_table**: Table name for which you want to create a facility type table. For our case it is one of the health facility datasets you've explored previously. Ideally you should use the HF dataset that you modified with standardized column names. The table file format should be csv or xlsx

* **facility_name**: Column name in the input table (aka health facility dataset) for the column that has facility name. If using the HF dataset version that you modified with column name standardization, you would put "facility_name" here

*   **type_dictionary_table**: type dictionary table that you created from the previous python exercise

*   **spelling_dictionary_table**: spelling dictionary table that you created from the previous python exercise





In [ ]:
country_name="NGA"
health_facility_table="eHealth_HFL_OpenAfrica_modified.xlsx"
facility_name="facility_name"
spelling_dictionary_table="merged_spelling_dictionary_v3.xlsx"
type_dictionary_table="Merged_type_dictionary_v3.xlsx"


# Part 3: Upload input table

The cell below allows you to choose the input table from your computer. After you run the cell below, you should see a **choose files** option. Click on the **choose files** option and then navigate your computer to find the files you want to read as **health_facility_table** (aka the health facility dataset) and **spelling_dictionary_table** and **type_dictionary_table**  from a previous exercise. You need to choose all files at the same time. It may take some time for Python to read a table based on file size and speed of your internet connection.
 


In [ ]:
uploaded=files.upload()

Saving eHealth_HFL_OpenAfrica_modified.csv to eHealth_HFL_OpenAfrica_modified (1).csv
Saving merged_spelling_dictionary_v3.xlsx to merged_spelling_dictionary_v3 (2).xlsx
Saving Merged_type_dictionary_v3.xlsx to Merged_type_dictionary_v3 (2).xlsx


#Part 4: Create word frequency table

Run the cell below. No need for any modification. After processing is done, you should see a csv file downloaded to your computer. The file is going to be saved in your **downloads** directory. The output table will have the same name as the health facility dataset with cleaned at the end. For example, if the input table name is DRC_health_facility, the output table name is going to be DRC_health_facility_cleaned. It may take some time. Please wait until the process is fully complete.

In [ ]:
def preclean(df, input_variable, output_variable, remove_accent=False):
  '''
    Basic cleaning and standardization of a column
    :param df: dataframe
    :param input_variable: a column name to be cleaned/standardized
    :param output_variable: cleaned/standardized column name
    :param remove_accent:remove french characters
    :return: dataframe
  '''
    ##=============================================================================#
  df[output_variable] = df[input_variable] + " "
  # replace NAs with empty string ''
  df[output_variable] = df[output_variable].fillna('')
  # remove accent marks
  if remove_accent:
      df[output_variable] = [unidecode.unidecode(n) for n in df[output_variable]]
  df[output_variable] = df[output_variable] \
      .str.replace(" III | Iii  | iii ", " 3 ") \
      .str.replace(" II | Ii  | ii ", " 2 ") \
      .str.replace(" I | i ", " 1 ") \
      .str.replace(" IV | Iv | iv ", " 4 ") \
      .str.replace('&', 'and')
  df[output_variable] = df[output_variable].apply(lambda x: " ".join(re.split('(\d+)', x)))
  df[output_variable] = df[output_variable].map(lambda x: re.sub(r'[#$%&*+,-./:;<=>?@[\]^_`{|}~]', ' ', x)).str.strip().str.replace("  "," ")
  df[output_variable] = df[output_variable].str.title(). \
      str.replace(r'\s+', ' ').str.strip()
  # replace NAs in output_variable with empty string ''
  df[output_variable] = df[output_variable].fillna(' ')




def correct_spelling(df, spelling_dict, clean_name, output_col):
    '''
    Makes correction to possible misspellings using the spelling dictionary
    :param df: dataframe
    :param spelling_dict: csv file with correct spelling dictionary
    :param clean_name:cleaned/standardized column name
    :param output_col:A column name to hold corrected names
    :param spelling_dict: Country specific spelling dictionary
    :return:dataframe
    '''


    words_to_correct = spelling_dict['word'].unique()

    df[output_col] = df[clean_name]
    for word in words_to_correct:
        misspellings = list(spelling_dict[(spelling_dict['word'] == word)]['misspelling'])
        for misspelling in misspellings:
            df[output_col] = df[output_col] \
                .str.replace('|'.join(['^' + misspelling + ' ', ' ' + misspelling + ' ',
                                       ' ' + misspelling + '$', '^' + misspelling + '$']), ' ' + word + ' ',
                             case=False) \
                .str.strip().replace(" De ", " de ")


    # reset and drop index
    df.reset_index(inplace=True, drop=True)


def remove_type_info(df, type_dict, clean_name, clean_name_final):
    """
    Use facility type and abbreviations in the type dictionary as keywords and remove type information
    :param df: dataframe
    :param type_dict: Country specific type dictionary as csv file
    :param clean_name: Cleaned/standardized name column
    :param clean_name_final: Output column to keep name without a type
    :return: dataframe
    """
    # remove whitespace between abbreviations of length 2 or 3
    # e.g. change C S to CS

    # obtain abbreviations of length 2 or 3
    type_dict['abbreviation'].fillna(type_dict['type'], inplace=True)
    tmp = type_dict[type_dict['abbreviation'].str.len() <= 4]['abbreviation'].unique()
    # sort by decreasing length
    tmp = sorted(tmp, key=len, reverse=True)
    # change it to the pattern '^c s ' or ' c s$'
    tmp_dict = {}
    for t in tmp:
        tmp_dict[t] = ['^' + ' '.join(list(t)) + ' ', ' ' + ' '.join(list(t)) + '$']
    # replace the pattern with 'cs'
    for t in tmp:
        pats = tmp_dict[t]
        df[clean_name] = df[clean_name].str.replace(pats[0], t + ' ', case=False) \
            .str.replace(pats[1], ' ' + t, case=False)

    # facility types
    types = list(type_dict['type'])
    type_keywords = set()
    for t in types:
        # add the full facility type
        t = t.title()
        type_keywords.add(t)

        # add individual words as well
        t = t.replace('/', ' ')
        words = t.split(' ')
        # skip words that have punctuation / numbers and have length <= 3 (e.g. de, (major))
        words = [w for w in words if w.isalpha() and len(w) > 3]
        for w in words:
            type_keywords.add(w)

    # obtain the list of type keywords and sort in descending length
    type_keywords = list(type_keywords)
    type_keywords = sorted(type_keywords, key=lambda s: -len(s))

    # abbreviations for that country
    abbrevs = set(type_dict['abbreviation'])

    abb_keywords = []
    for abbrev in abbrevs:
        # e.g. for CS, 4 patterns are considered: '^CS ', ' CS ', ' CS$', '^CS$'
        abbrev = abbrev.title()
        abb_keywords.extend(['^' + abbrev + '\s', '\s' + abbrev + '\s', '\s' + abbrev + '$',
                             '^' + abbrev + '$'])

    # obtain the list of abbreviation keywords and sort in descending length
    abb_keywords = sorted(abb_keywords, key=lambda s: -len(s))

    # handle situations when type is 'Hospital District' in the type dictionary
    # but name column has 'District Hospital' in ISS data
    type_len_2 = [t for t in type_keywords if len(t.split()) == 2]
    for t in type_len_2:
        df[clean_name] = df[clean_name].str.title() \
            .str.replace(' '.join(t.split()[::-1]), t, case=False)

    # remove type information using keywords generated above
    # remove meaningless connecting words like de, do, da, du
    df[clean_name_final] = df[clean_name].str.title() \
        .str.replace('|'.join(type_keywords), '') \
        .str.replace('|'.join(abb_keywords), ' ') \
        .str.strip() \
        .str.replace('^de | de | de$|^de$|^do | do | do$|^do$|^da | da | da$|^da$|^du | du | du$|^du$ |^dos|^das',
                     ' ', case=False) \
        .str.replace("  ", " ") \
        .str.strip() \
        .str.title()

def extract_type(df, clean_name, clean_name_final, extract_type):
    '''
    Remove type of a facility from a facility name, and puts type into a separate column
                 #exp: Kayamba Centre de Sante >> Kayamba  ||  Centre de Sante
    :param df: dataframe
    :param clean_name: cleaned/standardized column name
    :param clean_name_final: A column name that holds only name(no type)
    :param extract_type: A column name that holds only type
    :return: dataframe
    '''
    extract_types = []
    df.clean_name.fillna("", inplace=True)
    df[clean_name_final].fillna("", inplace=True)
    for idx, row in df.iterrows():
        name = row[clean_name].upper()
        name_final = row[clean_name_final].upper()

        # if clean_name_final is exactly the same as clean_name,
        # this indicates no type information can be extracted, thus append NA
        if name.upper() == name_final.upper():
            extract_types.append(np.nan)

        else:
            name = OrderedSet(name.split())
            name_final = OrderedSet(name_final.split())
            # find the difference between two names
            diff = ' '.join(list(name.difference(name_final)))
            extract_types.append(diff.strip())

    # remove de, do, da, du at start or end of extract_type
    # replace empty string with NA
    df[extract_type] = extract_types
    df[extract_type] = df[extract_type].str.strip() \
        .str.replace("  ", " ") \
        .str.replace('^de |^do |^da |^du | du$| de$| do$| da$|^de$|^do$|^da$|^du$', '', case=False) \
        .str.replace('^de |^do |^da |^du | du$| de$| do$| da$|^de$|^do$|^da$|^du$', '', case=False) \
        .str.strip() \
        .str.title() \
        .replace('', np.nan)
    # replace empty string with NA
    df[clean_name].replace('', np.nan, inplace=True)
    df[clean_name_final].replace('', np.nan, inplace=True)

def map_type(df, extract_type, sub_type, score, type_dict):
    '''
    Function=Use extract_type to map the type information extracted from the name column to one of the types in the type dictionary.
    :param df: dataframe
    :param extract_type: Type that extracted from name
    :param sub_type: A column name to hold type
    :param score: A column name to hold match score between extracted type and type dictionary
    :param type_dict:  Country specific type dictionary
    :return: dataframe
    '''

    types, abbrevs = type_dict['type'], type_dict['abbreviation']
    sub_types = []
    scores = []

    for idx, row in df.iterrows():
        # if extract_type is NA, just append NA
        if not isinstance(row[extract_type], str):
            sub_types.append(np.nan)
            scores.append(np.nan)

        # find best match
        else:
            match, match_score = process.extractOne(row[extract_type], list(types) + list(abbrevs),
                                                    scorer=fuzz.ratio)

            scores.append(match_score)
            # if best match is abbreviation, map it to the corresponding type
            if match in list(abbrevs):
                match_type = type_dict[type_dict['abbreviation'] == match]['type'].iloc[0]
                sub_types.append(match_type)
            else:
                sub_types.append(match)
    df[sub_type] = sub_types
    df[score] = scores

def basic_checks(df, facility_name_clean, facility_type_clean):
    '''
    add four new column to input df to keep lenght of facility name, lenght of the type, if
    facility name includes any special characters, and facility name with only numeric characters
    :param df:
    :return: add these columns to input df
    hf_name_lenght: lenght of facility name, Too long (30>) and short (<3) name needs a closer look
    hf_type_lenght: lenght of facility name, Too long (30>) and short (<3) name needs a closer look
    has_special_chrs: list of the special character a facility name has
    only_numeric: facilities with only numerical values
    '''
    df["hf_name_lenght"] = ""
    df["hf_type_lenght"] = ""
    df["special_chrs"] = ""
    df["only_numeric"] = ""
    df[facility_name_clean].fillna("", inplace=True)
    df[facility_type_clean].fillna("", inplace=True)
    for index, row in df.iterrows():
        df.loc[index, "hf_name_lenght"] = len(row[facility_name_clean])
        df.loc[index, "hf_type_lenght"] = len(row[facility_type_clean])

    for index, row in df.iterrows():
        check_list = ["?", "!", "//", "\\", "*", "!", "~", ")", "(", "&", "$", "+", "^", "%", "√¥", "√à","`",
                      "√ßo", "√®", "√©"]
        for i in check_list:
            if i in row[facility_name_clean]:
                df.loc[index, "special_chrs"] = row["special_chrs"] + "" + i

        if row[facility_name_clean].isnumeric():
            df.loc[index, "only_numeric"] = "all_numerical"



### processing ###

# read health facility table
if health_facility_table.endswith(".csv"):  
  hf_df=pd.read_csv(io.BytesIO(uploaded[health_facility_table])) 
elif health_facility_table.endswith(".xlsx") :  
  hf_df=pd.read_excel(io.BytesIO(uploaded[health_facility_table])) 

# spelling dictionary table table
if spelling_dictionary_table.endswith(".csv"):  
  sd_df=pd.read_csv(io.BytesIO(uploaded[spelling_dictionary_table])) 
  sd_df=sd_df.drop_duplicates(['word', 'misspelling'])
elif spelling_dictionary_table.endswith(".xlsx") :  
  sd_df=pd.read_excel(io.BytesIO(uploaded[spelling_dictionary_table])) 
  sd_df=sd_df.drop_duplicates(['word', 'misspelling'])

# type dictionary table table
if type_dictionary_table.endswith(".csv"):  
  td_df=pd.read_csv(io.BytesIO(uploaded[type_dictionary_table])) 
  td_df=td_df.drop_duplicates(['type'])
elif type_dictionary_table.endswith(".xlsx") :  
  td_df=pd.read_excel(io.BytesIO(uploaded[type_dictionary_table])) 
  td_df=td_df.drop_duplicates(['type'])


preclean(hf_df, facility_name,"clean_name")
correct_spelling(hf_df,sd_df, "clean_name" ,"corrected_name")
remove_type_info(hf_df,td_df, "corrected_name", "facility_name_short")
extract_type(hf_df, "corrected_name", "facility_name_short","extract_type")
map_type(hf_df ,"extract_type", "sub_type", "score", td_df)
basic_checks(hf_df, "facility_name_short","sub_type")
hf_df.drop(["clean_name","corrected_name"], inplace=True, axis=1, errors="ignore")




#export type_df
output_table_name=health_facility_table.split(".")[0]+"_cleaned.xlsx"
hf_df.to_excel(output_table_name)
files.download(output_table_name)

KeyError: ignored

#Part 5: Repeat for all datasets and combine into one file

Repeat all steps for each health facility dataset you have explored in previous landscaping exercises. Upload the file after you have done a manual check as per the instructions above. Place the file in your country's health facilities folder on the Drive.

